<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -q
!wandb login

wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import keras.applications
import wandb
from wandb.keras import WandbCallback

In [ ]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [ ]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbign and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [ ]:
vgg19=tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_vgg19=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg19,
  layers.Flatten(),
  layers.Dense(1028, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
vgg19.trainable=False
model_vgg19.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 1028)              25791492  
                                                                 
 dense_3 (Dense)             (None, 5)                 5145      
                                                                 
Total params: 45,821,021
Trainable params: 25,796,637
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
model_vgg19.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [ ]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "vgg19_1",
      tags = "without augment without dropout"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [ ]:
epochs=40
history = model_vgg19.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 6.0837 - accuracy: 0.5861

wandb: Adding directory to artifact (/content/wandb/run-20221007_110319-10f8odav/files/model-best)... Done. 2.5s


19/19 [==============================] - 253s 8s/step - loss: 6.0837 - accuracy: 0.5861 - val_loss: 0.2840 - val_accuracy: 0.9122
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.1914 - accuracy: 0.9409

wandb: Adding directory to artifact (/content/wandb/run-20221007_110319-10f8odav/files/model-best)... Done. 2.5s


...

19/19 [==============================] - 16s 844ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0729 - val_accuracy: 0.9662
Epoch 10/40
19/19 [==============================] - 9s 432ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0790 - val_accuracy: 0.9527
Epoch 11/40
19/19 [==============================] - 9s 428ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0759 - val_accuracy: 0.9595
Epoch 12/40
19/19 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221007_110319-10f8odav/files/model-best)... Done. 2.5s


19/19 [==============================] - 16s 855ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0700 - val_accuracy: 0.9662
Epoch 13/40
19/19 [==============================] - 9s 431ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0740 - val_accuracy: 0.9662
Epoch 14/40
19/19 [==============================] - 9s 433ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0732 - val_accuracy: 0.9662
Epoch 15/40
19/19 [==============================] - 9s 440ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0751 - val_accuracy: 0.9595
Epoch 16/40
19/19 [==============================] - 9s 436ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0730 - val_accuracy: 0.9662
Epoch 17/40
19/19 [==============================] - 9s 431ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0722 - val_accuracy: 0.9662
Epoch 18/40
19/19 [==============================] - 9s 430ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0766 - val_accuracy: 0.9662
Epoch 19/4

wandb: Adding directory to artifact (/content/wandb/run-20221007_110319-10f8odav/files/model-best)... Done. 2.4s


19/19 [==============================] - 17s 889ms/step - loss: 4.1213e-04 - accuracy: 1.0000 - val_loss: 0.0700 - val_accuracy: 0.9730
Epoch 38/40
19/19 [==============================] - 9s 427ms/step - loss: 3.9521e-04 - accuracy: 1.0000 - val_loss: 0.0707 - val_accuracy: 0.9730
Epoch 39/40
19/19 [==============================] - 9s 434ms/step - loss: 3.8356e-04 - accuracy: 1.0000 - val_loss: 0.0719 - val_accuracy: 0.9730
Epoch 40/40
19/19 [==============================] - 9s 436ms/step - loss: 3.6514e-04 - accuracy: 1.0000 - val_loss: 0.0724 - val_accuracy: 0.9730


In [ ]:
wandb.finish()

accuracy,▁▇▇█████████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▆▆▇▇▆▆▇▆▆▇▇▇▆▇▇▇▇▇▇▇█▇█▇██▇███████████
val_loss,█▇▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,36
best_val_loss,0.06997
epoch,39
loss,0.00037
val_accuracy,0.97297
